In [40]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [41]:
df = pd.read_csv('files/datos_agregados_ciudades.csv', sep=';')

Los campos que contiene este dataset son:

- Nombre del municipio (municipios españoles de más de 75.000 habitantes)
- Provincia
- Comunidad Autónoma
- Población (fuente de datos: INE, 2022)
- Kilómetros de carril bici en el municipio por cada 100.000 habitantes (fuentes de datos: El País, La Sexta, ayuntamientos)
- Precios de viviendas en alquiler por metro cuadrado (fuente: Idealista, marzo 2023)
- Distancia a Madrid en km (fuente: Google Maps)
- Tiempo medio de viaje en coche a Madrid (fuente: Google Maps)
- Tiempo medio de viaje en tren o autobús a Madrid (fuente: Google Maps)
- Distancia a Barcelona en km (fuente: Google Maps)
- Tiempo medio de viaje en coche a Barcelona (fuente: Google Maps)
- Tiempo medio de viaje en tren o autobús a Barcelona (fuente: Google Maps)
- Aeropuerto internacional más cercano (fuentes: AENA, Google Maps)
- Número de conexiones del aeropuerto internacional más cercano (fuente: AENA)
- Distancia en km al aeropuerto internacional más cercano (fuente: Google Maps)
- Temperatura máxima media en ºC (fuente: Weatherspark)
- Temperatura mínima media en ºC (fuente: Weatherspark)
- Número de visitantes internacionales (turistas) al año (pendiente de completar)
- Precio medio de venta de viviendas por metro cuadrado (fuente: Idealista, marzo 2023)
- Patrimonio de la Humanidad: Si dispone o no de uno o varios Patrimonios (fuente: Wikipedia)
- Porcentaje de Días con calidad de aire “Buena” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Porcentaje de Días con calidad de aire “Razonablemente Buena” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Porcentaje de Días con calidad de aire “Regular” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Porcentaje de Días con calidad de aire “Desfavorable” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Porcentaje de Días con calidad de aire “Muy Desfavorable” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Porcentaje de Días con calidad de aire “Extremadamente Desfavorable” (fuente: Ministerio para la Transición Ecológica y Reto Demográfico)
- Ponderación calidad del aire: Cálculo propio a partir de los 6 valores anteriores
- Horas de sol medias anuales (fuente: AEMET)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 29 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Municipio                                                      101 non-null    object 
 1   Provincia                                                      101 non-null    object 
 2   Comunidad Autónoma                                             101 non-null    object 
 3   Habitantes (2022)                                              101 non-null    float64
 4   Carril Bici                                                    101 non-null    object 
 5   Precios alquiler                                               101 non-null    object 
 6   Distancia a Madrid (km)                                        101 non-null    float64
 7   Tiempo medio 
de viaje a Madrid 
en coche                     

Obj -> Float -- Carril Bici, Precios alquiler, Distancia al aeropuerto internacional más cercano (km), Precio medio vivienda (EUR/m2), % días calidad del aire Buena, % días calidad del aire Razonablemente Buena, % días calidad del aire Regular, % días calidad del aire Desfavorable, % días calidad del aire Muy Desfavorable, % días calidad del aire Extremadamente Desfavorable, Ponderación calidad del aire y Horas de Sol medias anuales

Tiempo -- Tiempo medio de viaje a Madrid en coche, Tiempo medio de viaje a Madrid en tren o autobús, Tiempo medio de viaje a Barcelona en coche y Tiempo medio de viaje a Barcelona en tren o autobús

In [43]:
df.drop_duplicates(keep=False, inplace=True)

In [46]:
# Datos de visitantes internacionales (anuales) proporcionados por ChatGPT
datos_visitantes = {
    "Madrid": 9900000,
    "Barcelona": 8200000,
    "Sevilla": 3000000,
    "Palma": 2400000,
    "València": 2060000,
    "Málaga": 1500000,
    "Bilbao": 1100000,
    "Córdoba": 929781,
    "Santiago de Compostela": 926153,
    "Alicante": 841152,
    "Marbella": 742172,
    "Salamanca": 701422,
    "Toledo": 593970,
    "Chiclana de la Frontera": 479477
}


In [47]:
df["Número de visitantes internacionales al año"] = df["Municipio"].map(datos_visitantes)


In [52]:
df.drop(columns='Unnamed: 28', inplace=True)

In [56]:
df['Patrimonio de la Humanidad'].fillna('No', inplace=True)

In [57]:
df.isna().sum()

Municipio                                                            0
Provincia                                                            0
Comunidad Autónoma                                                   0
Habitantes (2022)                                                    0
Carril Bici                                                          0
Precios alquiler                                                     0
Distancia a Madrid (km)                                              0
Tiempo medio \nde viaje a Madrid \nen coche                          0
Tiempo medio de viaje a Madrid en tren o autobús                     5
Distancia a Barcelona (km)                                           0
Tiempo medio \nde viaje a Barcelona \nen coche                       0
Tiempo medio de viaje a Barcelona en tren o autobús                  5
Aeropuerto Internacional más cercano                                 0
Número de\nconexiones del\naeropuerto internacional\nmás cercano     0
Distan

In [59]:
# Define a function to clean numeric columns (replace commas and handle non-numeric values)
def clean_numeric_column(column):
    return column.replace(",", ".").replace("N/D", "").strip()

# Columns to convert to float
float_columns = [
    'Carril Bici',
    'Precios alquiler',
    'Distancia al aeropuerto internacional más cercano (km)',
    'Precio medio vivienda (EUR/m2)',
    '% días calidad del aire Buena',
    '% días calidad del aire Razonablemente Buena',
    '% días calidad del aire Regular',
    '% días calidad del aire Desfavorable',
    '% días calidad del aire Muy Desfavorable',
    '% días calidad del aire Extremadamente Desfavorable',
    'Ponderación calidad del aire',
    'Horas de Sol medias\nanuales'
]

# Clean and convert the columns to float
for column in float_columns:
    if column in df.columns:
        df[column] = df[column].astype(str).apply(clean_numeric_column)
        df[column] = pd.to_numeric(df[column], errors='coerce')

# Convert time columns to a consistent numerical format (e.g., minutes)
def time_to_minutes(time_str):
    if isinstance(time_str, str):
        if ":" in time_str:
            # Format hh:mm
            h, m = map(int, time_str.split(":"))
            return h * 60 + m
        elif "h" in time_str and "min" in time_str:
            # Format xh ymin
            h, m = time_str.split("h")
            m = m.replace("min", "").strip()
            return int(h.strip()) * 60 + int(m.strip())
    return None

time_columns = [
    'Tiempo medio \nde viaje a Madrid \nen coche',
    'Tiempo medio de viaje a Madrid en tren o autobús',
    'Tiempo medio \nde viaje a Barcelona \nen coche',
    'Tiempo medio de viaje a Barcelona en tren o autobús'
]

# Clean and convert time columns to minutes
for column in time_columns:
    if column in df.columns:
        df[column] = df[column].apply(time_to_minutes)

# Display updated DataFrame information
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 100
Data columns (total 28 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Municipio                                                      101 non-null    object 
 1   Provincia                                                      101 non-null    object 
 2   Comunidad Autónoma                                             101 non-null    object 
 3   Habitantes (2022)                                              101 non-null    float64
 4   Carril Bici                                                    77 non-null     float64
 5   Precios alquiler                                               100 non-null    float64
 6   Distancia a Madrid (km)                                        101 non-null    float64
 7   Tiempo medio 
de viaje a Madrid 
en coche                      101 

In [60]:
# Impute missing values based on column types and context
# Numeric columns: use mean or median depending on skewness
numeric_cols = df.select_dtypes(include=['float64']).columns

for col in numeric_cols:
    if col in df.columns:
        # Use mean for general cases, or median if skewed data is suspected (e.g., highly variable scales)
        df[col].fillna(df[col].median(), inplace=True)

# Display info to verify that null values have been handled
df.info(), df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 100
Data columns (total 28 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Municipio                                                      101 non-null    object 
 1   Provincia                                                      101 non-null    object 
 2   Comunidad Autónoma                                             101 non-null    object 
 3   Habitantes (2022)                                              101 non-null    float64
 4   Carril Bici                                                    101 non-null    float64
 5   Precios alquiler                                               101 non-null    float64
 6   Distancia a Madrid (km)                                        101 non-null    float64
 7   Tiempo medio 
de viaje a Madrid 
en coche                      101 

C:\Users\ariza\AppData\Local\Temp\ipykernel_12804\1165229830.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


(None,
 Municipio                                                           0
 Provincia                                                           0
 Comunidad Autónoma                                                  0
 Habitantes (2022)                                                   0
 Carril Bici                                                         0
 Precios alquiler                                                    0
 Distancia a Madrid (km)                                             0
 Tiempo medio \nde viaje a Madrid \nen coche                         0
 Tiempo medio de viaje a Madrid en tren o autobús                    0
 Distancia a Barcelona (km)                                          0
 Tiempo medio \nde viaje a Barcelona \nen coche                      0
 Tiempo medio de viaje a Barcelona en tren o autobús                 0
 Aeropuerto Internacional más cercano                                0
 Número de\nconexiones del\naeropuerto internacional\nmás cercano    0

In [61]:
df.head()

,Municipio,Provincia,Comunidad Autónoma,Habitantes (2022),Carril Bici,Precios alquiler,Distancia a Madrid (km),Tiempo medio \nde viaje a Madrid \nen coche,Tiempo medio de viaje a Madrid en tren o autobús,Distancia a Barcelona (km),Tiempo medio \nde viaje a Barcelona \nen coche,Tiempo medio de viaje a Barcelona en tren o autobús,Aeropuerto Internacional más cercano,Número de\nconexiones del\naeropuerto internacional\nmás cercano,Distancia al aeropuerto internacional más cercano (km),Temperatura máxima media (ºC),Temperatura mínima media (ºC),Número de visitantes internacionales al año,Precio medio vivienda (EUR/m2),Patrimonio de la Humanidad,% días calidad del aire Buena,% días calidad del aire Razonablemente Buena,% días calidad del aire Regular,% días calidad del aire Desfavorable,% días calidad del aire Muy Desfavorable,% días calidad del aire Extremadamente Desfavorable,Ponderación calidad del aire,Horas de Sol medias\nanuales
0,A Coruña,A Coruña,Galicia,244700.0,20.4,9.1,593.0,337,272.0,1084.0,370.0,284.5,A Coruña,9.0,11.6,24.0,7.0,1014890.5,2.300,Sí,20.0,73.0,4.0,3.0,0.0,0.0,410.0,2.453
1,Albacete,Albacete,Castilla-La Mancha,172357.0,24.4,7.1,260.0,154,97.0,545.0,322.0,325.0,Alacant-Elx \nMiguel Hernández,119.0,169.0,33.0,0.0,1014890.5,1.417,No,3.0,72.0,16.0,9.0,0.0,0.0,369.0,3.282
2,Alcalá de Guadaíra,Sevilla,Andalucía,75917.0,19.4,6.6,531.0,309,209.0,995.0,590.0,284.5,Sevilla,70.0,21.1,35.0,5.0,1014890.5,1.715,No,14.0,72.0,11.0,3.0,0.0,0.0,397.0,3.526
3,Alcalá de Henares,Madrid,Madrid,196888.0,28.0,9.9,40.0,39,228.0,586.0,330.0,200.0,Adolfo Suárez \nMadrid-Barajas,174.0,21.0,33.0,1.0,1014890.5,1.715,Sí,17.0,68.0,11.0,3.0,0.0,0.0,396.0,2.691
4,Alcobendas,Madrid,Madrid,117041.0,53.8,12.0,24.0,25,228.0,613.0,347.0,229.0,Adolfo Suárez \nMadrid-Barajas,174.0,9.1,33.0,0.0,1014890.5,1.715,No,22.0,67.0,8.0,3.0,0.0,0.0,408.0,2.691


In [63]:
df.columns = df.columns.str.replace("\n", "")

In [70]:
df['Aeropuerto Internacional más cercano'] = df['Aeropuerto Internacional más cercano'].str.replace("\n", "")

In [71]:
df.head()

,Municipio,Provincia,Comunidad Autónoma,Habitantes (2022),Carril Bici,Precios alquiler,Distancia a Madrid (km),Tiempo medio de viaje a Madrid en coche,Tiempo medio de viaje a Madrid en tren o autobús,Distancia a Barcelona (km),Tiempo medio de viaje a Barcelona en coche,Tiempo medio de viaje a Barcelona en tren o autobús,Aeropuerto Internacional más cercano,Número deconexiones delaeropuerto internacionalmás cercano,Distancia al aeropuerto internacional más cercano (km),Temperatura máxima media (ºC),Temperatura mínima media (ºC),Número de visitantes internacionales al año,Precio medio vivienda (EUR/m2),Patrimonio de la Humanidad,% días calidad del aire Buena,% días calidad del aire Razonablemente Buena,% días calidad del aire Regular,% días calidad del aire Desfavorable,% días calidad del aire Muy Desfavorable,% días calidad del aire Extremadamente Desfavorable,Ponderación calidad del aire,Horas de Sol mediasanuales
0,A Coruña,A Coruña,Galicia,244700.0,20.4,9.1,593.0,337,272.0,1084.0,370.0,284.5,A Coruña,9.0,11.6,24.0,7.0,1014890.5,2.300,Sí,20.0,73.0,4.0,3.0,0.0,0.0,410.0,2.453
1,Albacete,Albacete,Castilla-La Mancha,172357.0,24.4,7.1,260.0,154,97.0,545.0,322.0,325.0,Alacant-Elx Miguel Hernández,119.0,169.0,33.0,0.0,1014890.5,1.417,No,3.0,72.0,16.0,9.0,0.0,0.0,369.0,3.282
2,Alcalá de Guadaíra,Sevilla,Andalucía,75917.0,19.4,6.6,531.0,309,209.0,995.0,590.0,284.5,Sevilla,70.0,21.1,35.0,5.0,1014890.5,1.715,No,14.0,72.0,11.0,3.0,0.0,0.0,397.0,3.526
3,Alcalá de Henares,Madrid,Madrid,196888.0,28.0,9.9,40.0,39,228.0,586.0,330.0,200.0,Adolfo Suárez Madrid-Barajas,174.0,21.0,33.0,1.0,1014890.5,1.715,Sí,17.0,68.0,11.0,3.0,0.0,0.0,396.0,2.691
4,Alcobendas,Madrid,Madrid,117041.0,53.8,12.0,24.0,25,228.0,613.0,347.0,229.0,Adolfo Suárez Madrid-Barajas,174.0,9.1,33.0,0.0,1014890.5,1.715,No,22.0,67.0,8.0,3.0,0.0,0.0,408.0,2.691


In [72]:
df.to_csv('files/101-ciudades-españa.csv')